UP206a Midterm by the Mangonadas: Andres and Dani
Our research question: Are communities that are proximate to environmental health hazards more policed than those not exposed to environmental health hazards?

This project intends to examine the intersections between proximity to environmental hazards and policng because environmental risks could indicate increased susceptibility to developing sociobehavioral issues deemed as deviant.

#This question is important because polluted communities could be doubly vulnerable given their immediate social and physical conditions, making them subject to policing of their behaviors.

#To explore our question, we are using census tract data from 2012, the LA City Geohub Environmental Justice Screening Method data, and crime data, to examine reports of arrest in Los Angeles. 

In [ ]:
#Import packages
import geopandas as gpd
import pandas as pd
import plotly.express as px
from sodapy import Socrata
import folium
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
#Read in EJSM df and make string of tracts to be named
df = pd.read_csv(
'Data/EJSM_Scores/EJSM_Scores (1).csv' ,
dtype={
    'Tract_1':str
})

In [ ]:
#Add number zero leading the FIPS code for merging the data with the census tract data
df['Tract_1'] = df['Tract_1'].str.zfill(11)
df.head()

In [ ]:
#Import 2012 census data
tracts=gpd.read_file('Data/CensusData2012/census-tracts-2012.geojson')
print(tracts)

In [ ]:
#List column names 
list(tracts)

In [ ]:
#Select unwanted columns
columns_to_drop = ['set','kind','resource_uri','metadata']
#read columns 
tracts.head()

In [ ]:
#Drop unnecessary columns from tracts data
tracts = tracts.drop(columns_to_drop,axis=1)

In [ ]:
#Show tracts info
tracts.info(verbose=True, null_counts=True)

In [ ]:
#Isolate the FIPS code and geometry columns to match with the EJSM data
tracts = tracts[['name','geometry']]
tracts.head()

In [ ]:
#Show isolated columns
tracts.columns = ['FIPS','geometry']
tracts.head()

In [ ]:
#Rename Tract_1 to FIPS
df.columns = ['OBJECTID',
 'FIPS',
 'CIscore',
 'HazScore',
 'HealthScore',
 'SVscore',
 'CCVscore',
 'Shape__Area',
 'Shape__Length']
df.head()

In [ ]:
#This section will explore the merged data

In [ ]:
#List new df columns
list(df)

In [ ]:
#Merge data on the same object FIPS
tracts_ejsm=tracts.merge(df,on="FIPS")
#Show merge with census data
tracts_ejsm.head()
tracts_ejsm.info()

In [ ]:
tracts_ejsm.crs

In [ ]:
#Describe stats by proximity to hazard score
tracts_ejsm['CIscore'].describe()

In [ ]:
#Plot histogram for cumulative impact score
tracts_ejsm['CIscore'].plot.hist(bins=50),
plt.xlabel('Cumulative Impact Scores')
plt.ylabel('Frequency'),
plt.savefig('CIscore.png')

In [ ]:
#Plot histogram for proximity to hazard score. It is grouped by the number of occurences. 
#Our not so great graph! Seeing this bar chart intrigued us to durther explore the variations between census tracts.
tracts_ejsm['HazScore'].plot.hist(bins=50)
plt.xlabel('Proximity to Hazards')
plt.ylabel('Frequecy'),
plt.savefig('HazScore_bar.png')

It’s unclear why the hazard score data is not distributed the same way as the cumulative impact score data. We wondered if it was because we had miswritten it or if the hazard occurrences were not visible on this bar graph. What is apparent is the average cumulative impact score of LA lying somewhere between 10.5-12.5. The cumulative score considers all the scores identified from the dataset included proximity to hazards, social vulnerability, climate vulnerability, and health vulnerability. 

In [ ]:
#New object to show the sorted data- these are the top polluted census tracts. The census tracts that are shown as being the most polluted are also the most socially vulnerable.
tracts_ejsm_sorted = tracts_ejsm.sort_values(by='CIscore',ascending = False)
tracts_ejsm_sorted[['FIPS','CIscore','HazScore','SVscore']].head(10)

In [ ]:
#Plot represents the proximity to environmental hazards score by equal intervals 
#We can see that the most polluted areas are in south central LA county 
tracts_ejsm.plot(figsize=(24,20),
                 column='HazScore',
                 legend=True, 
                 scheme='equal_interval')

In [ ]:
#Plot represents the cumulative impact scores separated by natural breaks
#The map shows increased variation because of the integrated scores
tracts_ejsm.plot(figsize=(24,20),
                 column='CIscore',
                 legend=True, 
                 scheme='NaturalBreaks')

In [ ]:
#Folium map for HazScore. This score will be our main point of analysis

m = folium.Map(location=[34.2,-118.2], 
               zoom_start = 9,
               tiles='CartoDB positron', 
               attribution='CartoDB')

# plot chorpleth over the base map
folium.Choropleth(
                  geo_data=tracts_ejsm, # geo data
                  data=tracts_ejsm, # data          
                  key_on='feature.properties.FIPS', # key, or merge column
                  columns=['FIPS', 'HazScore'], # [key, value]
                  fill_color='BuPu',
                  line_weight=0.1, 
                  fill_opacity=0.8,
                  line_opacity=0.2, # line opacity (of the border)
                  legend_name='Degree of proximity to Environmenta Hazards)').add_to(m)
m


This Chloropleth map provides us with a background map that will kick off the rest of our work. We chose hazard scores because while the cumulative impact was meaningful, we wanted to focus on a standard that could be more decisively measured (proximity to environmental hazards in the form of pollutants) rather than other measurements. Measurements include a social vulnerability that could be more skewed based on indicators defined by the source (in this case, USC/Occidental College). For example, although poverty and policing can be linked, we wanted to see if sites that were contaminated or close to toxic sites (such as Exide) would have a significant relationship to policing.

<div class="alert alert-danger">
There is a problem with the way you are importing the data. You have a limit of 10,000 records (this can be upped to 50,000), and you do not sort the data in any way. The resulting data is a subset of records of the total.
    
Consider instead to specify a date range and a query that makes the data fall under the 50000 record limit.

I provide an example below (df2) for your reference.
</div>

In [ ]:
#Importing our arrest 2020 data using an API
client = Socrata("data.lacity.org", None)
results = client.get("amvf-fr72", limit=10000)
df2 =pd.DataFrame.from_records(results)

In [ ]:
# importing a more complete dataset
results2 = client.get("amvf-fr72", 
                     limit=50000,
                     where = "(arst_date between '2020-01-01T00:00:00' and '2020-10-30T00:00:00') AND (grp_description in ('Aggravated Assault','Miscellaneous Other Violations','Driving Under Influence Other Assaults','Narcotic Drug Laws', 'Disturbing the Peace', 'Disorderly Conduct'))",
                     order='arst_date desc')

df2 =pd.DataFrame.from_records(results2)

In [ ]:
df2.shape

<div class="alert alert-danger">
This example limits the import to certain crime types (look at the list within the where statement). Adjust these as necessary.
</div>

In [ ]:
#Sample data to check for inconsistencies
df2.sample(7)

In [ ]:
#Get description of arrest data 
df2.grp_description.unique()

In [ ]:
#Examining all data types in the df for floats
df2.info()

<div class="alert alert-danger">
As you can see, the plot below paints a different picture.
</div>

In [ ]:
#Plotly bar graph to examine the total arrests by LAPD in 2020
px.bar(df2,
       x='arst_date',
       title='LAPD Arrests in 2020',
       labels={'arst_date':'Arrest date','counts':'Number of arrests'}
      )

This has been a graph circulating, but it’s interesting to note how arrests boom in the summer months and the moments leading up to it due to the uprisings. What would these look like across time? When hearing about the ‘92 uprising, a familiar anecdote is how heat and power shortages aggravated communities already frustrated and fed up with the policing system. While it’s hard to test this, it’s important to note how many people, particularly folks of color, we’re stuck at home due to quarantine during summer, and many lacked air conditioning (or income to pay for utilities).

In [ ]:
#Index arrest charge value counts
arrest_by_charge = df2.grp_description.value_counts().reset_index()
arrest_by_charge

<div class="alert alert-danger">
Again, these numbers will change with a more accurate import.
</div>

In [ ]:
#Index arrest charge value counts
arrest_by_charge = df2.grp_description.value_counts().reset_index()
arrest_by_charge

In [ ]:
#make lat and lon floats 
df2['lat'] = df2['lat'].astype(float)
df2['lon'] = df2['lon'].astype(float)

In [ ]:
df2.info()

In [ ]:
#Configure geometry for df2 (crime) geodataframe
df2 = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))

In [ ]:
#Scatterplot of crime data
df2.plot(figsize=(12,12),color='purple')

In [ ]:
#Add base from tracts data
base = tracts_ejsm.plot(figsize=(12,10),color='gainsboro', edgecolor='white')

ax = df2.plot(ax=base, color='purple', markersize=5)

In [ ]:
#Print total bounds of data points 
minx, miny, maxx, maxy = df2.geometry.total_bounds
print(minx)
print(maxx)
print(miny)
print(maxy)

In [ ]:
#Print map with within closer boundaries df2
base = tracts_ejsm.plot(figsize=(12,12),color='gainsboro', edgecolor='white')
ax = df2.plot(ax=base, marker='o', color='purple', markersize=5)
ax.set_xlim(minx - .1, maxx + .1)
ax.set_ylim(miny - .1, maxy + .1)
ax

In [ ]:
#Configure coordinate reference system for tracts to join the two dataframes
tracts.crs

In [ ]:
#Configure coordinate reference system for crime
df2.set_crs(epsg=4326, inplace=True)
df2.crs

In [ ]:
#Join tracts_ejsm and crime data
join = gpd.sjoin(tracts,
                 df2,
                 how='right')

In [ ]:
join.head()

In [ ]:
join.sample(5)

In [ ]:
join.shape

In [ ]:
#Subset the data from join
df2 = pd.DataFrame(join,columns=['arst_date','rpt_id','grp_description','arst_typ_cd','HazScore', 'CIscore','FIPS','lat','lon','geometry'])
print(df)

In [ ]:
#Subset of the joined dataframes- need to figure out what went wrong in this subset
#We are keeping these columns because our analysis focuses on the HazScore and description of the charges
df2.head()

Here we follow Yoh’s steps more closely, but the aggravated arrests don’t tell us much. We’re trying to figure out crimes linked explicitly to environmental hazards. For now, we want to keep this large dataset because we want to be able to pick them out later depending on what crimes we want to analyze in the nexus with environmental hazards.

In [ ]:
#These are the tracts with the highest number of arrests. We included this as a priliminary step to mapping the full counts. 
crime_by_tracts = join.FIPS.value_counts().rename_axis('Tract').reset_index(name='crime_count')
crime_by_tracts.head()

df2=

In [ ]:
#This is a visualization to compare the amount of arrests in different census tracts; df2 using yoh's subset
crime_by_tracts[:50].plot.bar(figsize=(20,8),x='Tract',y='crime_count')

This bar graph tells us the crime by census tract, but we can only guess what that is. It would be useful to bring this into the function, but we’re looking for a better way of organizing and visualizing the data. Instead of looking at specific census tracts themselves, our goal is to understand the interconnected relationship between proximity to hazards (such as pollution) and seeing what charges the police are giving people in that area. Surprisingly, the census tract with the most crimes (06037980028) is the one belonging to the airport and in between the Chevron Oil Refinery and the Ballon Wetlands


In [ ]:
#DF IS MISSING FIPS- Grouped data by the description of arrest. Here, the FIPS out of order because we are descibing a full count of charge descriptions. 
df.groupby(['grp_description']).count()

In [ ]:
#NEED TO REVISIT Group data by the FIPS code to explore the meaning of the counts in each census tract. 
df.groupby(['FIPS']).count()

In [ ]:
#Data is grouped for counts of reports in tracts. 
df_grouped=df.groupby(['FIPS','grp_description','lat','lon','arst_date']).count()[['rpt_id']]
df_grouped.head(50)

In [ ]:
#Flatten data to prep for bar graph and reset the index to include the group values 
df_flat = df_grouped.reset_index()
df_flat

Look at how beautiful that index is. What is not shown in our flattened data frame is the amount of blood and tears (and lots of green tea caffeine) that helped us get to this point. Originally while our workflow was simple, things became more complicated as we tried to clean up our notebooks without taking away code that would serve us moving forward. This data frame allows us to see all types of crimes and arrest data across census tracts and their location. This provides us with the top-most layer for our future maps, in addition to a versatile data frame that can give temporal nuance to our maps, an attribute especially useful for the Kepler map. 

<div class="alert alert-danger">
Modified the code below (added marker_line_width=0) to get rid of the white horizontal lines that were making the bars difficult to read.
</div>

In [ ]:
#Make plotly bar graph
fig = px.bar(df_flat,
       x='grp_description',
       y='rpt_id',
       title='Description of LAPD Arrests in 2020',
       color='grp_description',
       labels={'grp_description':'Arrest Decription','rpt_id':'Number of Arrests'}
      )
fig.update_traces(marker_line_width=0)

So far, so good. It was tricky to make sense of all these numbers, but having everything cleanly indexed would provide us more than enough for what we needed. Looking back now, we would like to have examined a larger sample, but based on our gigabyte limit on Jupyter Hub, we were unsure how far to push this. From this bar graph, it becomes clear that aggravated assault, driving under the influence, and other crimes are the largest ones. We’re trying to focus on aggravated assault, disturbing the peace, and disorderly contact, because these are the ones we believe can connect policing to proximity to hazards (the ejsm score). It’s clear that aggravated assault is the largest sample; there are significantly less for disorderly conduct and disturbing the peace. For now, we don’t know if this will affect the statistical significance of our findings later on. 

In [ ]:
#Define the merged dataframe
df_ejsm = tracts_ejsm.merge(df_flat,on="FIPS")
df_ejsm.info()

In [ ]:
#Convert lat and lon to floats intergrating arrest and EJSM score data
df_ejsm['lat'] = df_ejsm['lat'].astype(float)
df_ejsm['lon'] = df_ejsm['lon'].astype(float)
df_ejsm.info()

In [ ]:
#Convert dataframe to crs
df_ejsm.crs

In [ ]:
#Create basic scatter of df_ejsm_crime_tracts data with coordinate reference system
px.scatter(df_ejsm,
           x='lon',
           y='lat'
          )

In [ ]:
#Exploring map styles using mapbox
fig = px.scatter_mapbox(df_ejsm,
                        lat='lat',
                        lon='lon',
                        mapbox_style="stamen-terrain")
fig.show()

In [ ]:
#Make temporal scatter plot map
fig = px.scatter_mapbox(df_ejsm, 
                        lat="lat", 
                        lon="lon", 
                        color="grp_description",
                        animation_frame = 'arst_date',
                       )
fig.update_layout(mapbox_style="carto-darkmatter")

fig.show()

In [ ]:
#import kepler 
from keplergl import KeplerGl

In [ ]:
#make base map from reference
map = KeplerGl(height=600,width=800)
map

In [ ]:
#tracts_ejsm have no attribute string
map.add_data(data=df_ejsm,name='EJSM')

#At this point, we were stumped. We were clueless as to how our Kepler map was not fully functioning. In previous iterations, we were able to add on our merged data, but in this case, something has occurred in between our workflows where we couldn’t figure out the discrepancy. The first time around, we thought the geometry was converted to an object instead of remaining as geometry, and we were able to correct this. But even then, Kepler went unchanged. Though it had been simple, something occurred when we merged notebooks, and we still haven’t figured out why df_ejsm won’t work on Kepler even though it has a geometry file and the coordinates have been converted appropriately. After we went back, we realized we had merged the wrong tract files, and once we corrected ejsm_tracts to tracts, the rest of the notebook fell into place.  

In [ ]:
import contextily as ctx

In [ ]:
census_tracts = tracts_ejsm[['FIPS','HazScore','geometry']]
census_tracts.head()
census_tracts.info()

In [ ]:
census_tracts = census_tracts.to_crs(epsg=3857)

In [ ]:
#Use census tracts and ejsm integrated layers to add base map
ax=census_tracts.plot(figsize=(20,20), 
                      edgecolor='white',
                      column='HazScore',
                      alpha=0.5)

ax.axis('off')

ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

In [ ]:
#Make crime GeoDataFrame
crime = gpd.GeoDataFrame(df_flat, 
                         crs='EPSG:4326',
                         geometry=gpd.points_from_xy(df_flat.lon, df_flat.lat))

In [ ]:
#Convert crime to crs
crime = crime.to_crs(epsg=3857)
crime.info()

In [ ]:
#Use geometry to define area
crime.geometry.total_bounds

In [ ]:
#Print total bounds
minx, miny, maxx, maxy = crime.geometry.total_bounds
print(minx)
print(maxx)
print(miny)
print(maxy)

In [ ]:
#Configure function map by census tracts as the base map
base = census_tracts.plot(figsize=(20,20), 
                      edgecolor='white',
                      column='HazScore',
                      alpha=0.5)

ax = crime.plot(ax=base, marker='o', color='purple', markersize=5)

ax.set_xlim(minx - 7000, maxx + 7000) 
ax.set_ylim(miny - 7000, maxy + 7000)

ax.axis('off')

ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)


In [ ]:
#Define bounds for the map using the function
crime.geometry.total_bounds
minx, miny, maxx, maxy = crime.geometry.total_bounds
print(minx)
print(maxx)
print(miny)
print(maxy)

In [ ]:
def map_arrests_by_type(charge='Aggravated Assault'):
    
    crime_type = crime[crime.grp_description==charge]

    minx, miny, maxx, maxy = crime_type.geometry.total_bounds
    
    arrests_in_tracts = gpd.sjoin(census_tracts,crime_type,how='right')

    base = census_tracts.plot(figsize=(20,20), 
                    edgecolor='white',
                    column='HazScore',
                    alpha=0.5)

    ax = arrests_in_tracts.plot(ax=base, 
                                    column='grp_description',
                                    marker='o',
                                    markersize=40, 
                                    legend=True,
                                    cmap='tab20',
                                    legend_kwds={
                                       'loc': 'upper right',
                                       'bbox_to_anchor':(1.3,1)
                                    }                  # this puts the legend to the side
                                )

    ax.set_xlim(minx - 200, maxx + 200) # added/substracted value is to give some margin around total bounds
    ax.set_ylim(miny - 200, maxy + 200)

    ax.axis('off')

    ax.set_title('Occurences of '+crime.grp_description.values[0]+' in Los Angeles',fontsize=20)

    ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)
    ax

In [ ]:
#Map arrests by using the function
map_arrests_by_type(charge='Aggravated Assault')

At this point, we reached a dilemma. After extensive back and forth, we were unable to get the map to filter the crime points by type. Instead, it highlighted all the census tracts where these crimes had occurred. We had to stop and reassess our workflow since we weren’t making any progress and repeating previous issues. Our goal for our function was for the large map to show all the points we wanted so that rather than zooming in on a place, we instead see where crimes were committed by type and identify where certain crimes occurred more than others. Alas, we were still able to understand more. In previous iterations, we created a line of code using ".isin", though we also found that it aggregated the crimes we were trying to isolate.

<div class="alert alert-danger">
The issue with the map above was that you had the spatial join be an "inner" join, which prioritizes the census tract polygons (thus coloring them in). Instead, use "right", which prioritizes the crime data.
</div>

Division of labor
DH: Finalized notebooks, explored data, imported maps, coordinated with Andres to troubleshoot, indexed data and dataframes
AG: Visualized some of the interactive maps, including Kepler; co-ordinated with Dani to troubleshoot function; Multiple overlays